The goal of this exercise is to get into some first contact with Python, Pandas data frames, and the many opportunities how to quickly generate static charts from data frames. For the first exercise, load the provided CSV file, do some data cleaning, and check if your cleaning operation was successful using visualization.

The main purpose of the notebook is to preprocess the CSV file for further visualization. The following steps have to be performed: 

1. Read the three CSV using Pandas. See the pandas.read_csv documentation to check how to parse the CSV correctly! Merge the two datasets player_data_per_36_min.csv and players.csv into one table. Add a team_name column to your new data frame based on the team_id column retrieving the name from the teams.csv dataset. You should now have a table that includes player info from players.csv, and players performance statistics from player_data_per_36_min.csv, and the team’s name from teams.csv. (3 points)
2. Take care of missing values. Sci-kit learn provides different data imputation methods. Remove unusable rows or columns, if necessary.  Delete all observations for players who do not have a current team.
(Hint: You may need to impute before you completely filter your dataframe. You need to make an educated judgment). (3 points)
3. Create two new tables: one that groups rows based on player_id, another that groups rows based on team_id. (2 points)
4. Visualize the data (twice). Every submitted notebook should contain at least two visualizations using at least two different Python visualization libraries. One visualization for each data set (players and teams). A list of the most wide-spread Python visualization libraries can be found in this article. You must concisely describe and explain each visualization and your decisions in a Markdown field. You will not receive the points for this task if you did not add a description/explanation. (max. 5 points per visualization)
5. Save the resulting tables (players data, and aggregated teams data) as CSV. To be sure that the data is correctly saved, you can load it again. You will have to work with these table for the second exercise. (2 points)

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [3]:
players_df = pd.read_csv('data/players.csv')
teams = pd.read_csv('data/teams.csv')
player_data_df = pd.read_csv('data/player_data_per_36_min.csv')

In [4]:
merged_df = pd.merge(players_df, player_data_df, how='inner', left_on='id', right_on='player_id', suffixes=('_players', '_player_data'))

In [5]:
print(merged_df.head)

<bound method NDFrame.head of        id_players             name                full_name position height  \
0               1   Alaa Abdelnaby           Alaa Abdelnaby      F-C   6-10   
1               1   Alaa Abdelnaby           Alaa Abdelnaby      F-C   6-10   
2               1   Alaa Abdelnaby           Alaa Abdelnaby      F-C   6-10   
3               1   Alaa Abdelnaby           Alaa Abdelnaby      F-C   6-10   
4               1   Alaa Abdelnaby           Alaa Abdelnaby      F-C   6-10   
...           ...              ...                      ...      ...    ...   
26669        4814    Marial Shayok            Marial Shayok        G    6-5   
26670        4817  Jeremiah Martin          Jeremiah Martin       PG    6-3   
26671        4818   William Howard           William Howard        F    6-8   
26672        4819     Gabe Vincent   Gabriel Nnamdi Vincent       PG    6-3   
26673        4820   Marques Bolden   Marques Terrell Bolden        C   6-11   

       weight       b

In [6]:
print(merged_df.columns)

Index(['id_players', 'name', 'full_name', 'position', 'height', 'weight',
       'birth_date', 'birth_place', 'retired', 'player_url', 'current_team_id',
       'teams', 'text', 'id_player_data', 'player_id', 'season', 'total_games',
       'games_started', 'minutes_played', 'fg', 'fga', 'fgp', 'fg3', 'fg3a',
       'fg3p', 'fg2', 'fg2a', 'fg2p', 'ft', 'fta', 'ftp', 'orb', 'drb', 'trb',
       'ast', 'stl', 'blk', 'tov', 'pf', 'pts'],
      dtype='object')


In [7]:
merged_df = pd.merge(merged_df, teams, how='left', left_on='current_team_id', right_on='team_id')

In [9]:
print(merged_df.columns)

Index(['id_players', 'name', 'full_name', 'position', 'height', 'weight',
       'birth_date', 'birth_place', 'retired', 'player_url', 'current_team_id',
       'teams', 'text', 'id_player_data', 'player_id', 'season',
       'total_games_x', 'games_started', 'minutes_played', 'fg', 'fga', 'fgp',
       'fg3', 'fg3a', 'fg3p', 'fg2', 'fg2a', 'fg2p', 'ft', 'fta', 'ftp', 'orb',
       'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'team_id',
       'team_name', 'short_name', 'years', 'total_games_y', 'total_wins',
       'total_losses', 'win_loss_percentage', 'champions', 'other_names',
       'link', 'active'],
      dtype='object')


Task 2

In [10]:
imputer = SimpleImputer(strategy='mean')
merged_df[['games_started','minutes_played','fg','fga','fg3p','fg3','fg3a','fg2','fg2a', 'fg2p', 'ft','fta','orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']] = imputer.fit_transform(merged_df[['games_started','minutes_played','fg','fga','fg3p','fg3','fg3a','fg2','fg2a', 'fg2p', 'ft','fta','orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']])
merged_df = merged_df.dropna(subset=['current_team_id'])

In [11]:
print(merged_df)

       id_players             name                full_name position height  \
152            23     Steven Adams      Steven Funaki Adams        C   6-11   
153            23     Steven Adams      Steven Funaki Adams        C   6-11   
154            23     Steven Adams      Steven Funaki Adams        C   6-11   
155            23     Steven Adams      Steven Funaki Adams        C   6-11   
156            23     Steven Adams      Steven Funaki Adams        C   6-11   
...           ...              ...                      ...      ...    ...   
26669        4814    Marial Shayok            Marial Shayok        G    6-5   
26670        4817  Jeremiah Martin          Jeremiah Martin       PG    6-3   
26671        4818   William Howard           William Howard        F    6-8   
26672        4819     Gabe Vincent   Gabriel Nnamdi Vincent       PG    6-3   
26673        4820   Marques Bolden   Marques Terrell Bolden        C   6-11   

       weight       birth_date           birth_plac